In [1]:
import pandas as pd
import umap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import hdbscan

%matplotlib inline

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [3]:
from sklearn.cluster import KMeans

In [4]:
final_df = pd.read_csv("../data/processed/final_df.csv")

In [5]:
final_df.set_index('id_student', inplace=True)

In [6]:
def drop_col (df, list):
    df.drop(columns=(list), inplace=True)
    return df

In [7]:
col_drop = ["region", "gender", "press_mod", "code_presentation", "studied_credits", "highest_education"]

In [8]:
drop_col(final_df, col_drop)

,age_band,n_assignments,mean_score,final_result,total_clicks,code_module
id_student,,,,,,
11391,55<=,5,82.000000,Pass,934,AAA
28400,35-55,5,66.400000,Pass,1435,AAA
31604,35-55,5,76.000000,Pass,2158,AAA
32885,0-35,5,54.400000,Pass,1034,AAA
38053,35-55,5,68.000000,Pass,2445,AAA
...,...,...,...,...,...,...
2620947,0-35,9,88.888889,Distinction,476,GGG
2645731,35-55,9,88.111111,Distinction,893,GGG
2648187,0-35,9,76.666667,Pass,312,GGG


In [9]:
final_df['clicks_per_asmt'] = final_df['total_clicks']/final_df['n_assignments']



In [10]:
final_df

,age_band,n_assignments,mean_score,final_result,total_clicks,code_module,clicks_per_asmt
id_student,,,,,,,
11391,55<=,5,82.000000,Pass,934,AAA,186.800000
28400,35-55,5,66.400000,Pass,1435,AAA,287.000000
31604,35-55,5,76.000000,Pass,2158,AAA,431.600000
32885,0-35,5,54.400000,Pass,1034,AAA,206.800000
38053,35-55,5,68.000000,Pass,2445,AAA,489.000000
...,...,...,...,...,...,...,...
2620947,0-35,9,88.888889,Distinction,476,GGG,52.888889
2645731,35-55,9,88.111111,Distinction,893,GGG,99.222222
2648187,0-35,9,76.666667,Pass,312,GGG,34.666667


In [11]:
idx=final_df.index.tolist()

In [12]:
final_df.index.min()

6516

In [13]:
final_df

,age_band,n_assignments,mean_score,final_result,total_clicks,code_module,clicks_per_asmt
id_student,,,,,,,
11391,55<=,5,82.000000,Pass,934,AAA,186.800000
28400,35-55,5,66.400000,Pass,1435,AAA,287.000000
31604,35-55,5,76.000000,Pass,2158,AAA,431.600000
32885,0-35,5,54.400000,Pass,1034,AAA,206.800000
38053,35-55,5,68.000000,Pass,2445,AAA,489.000000
...,...,...,...,...,...,...,...
2620947,0-35,9,88.888889,Distinction,476,GGG,52.888889
2645731,35-55,9,88.111111,Distinction,893,GGG,99.222222
2648187,0-35,9,76.666667,Pass,312,GGG,34.666667


In [23]:
final_df.dtypes

age_band            object
n_assignments        int64
mean_score         float64
final_result        object
total_clicks         int64
code_module         object
clicks_per_asmt    float64
dtype: object

In [22]:
final_df.round({'mean_score': 1, 'clicks_per_asmt': 1})

,age_band,n_assignments,mean_score,final_result,total_clicks,code_module,clicks_per_asmt
id_student,,,,,,,
11391,55<=,5,82.0,Pass,934,AAA,186.8
28400,35-55,5,66.4,Pass,1435,AAA,287.0
31604,35-55,5,76.0,Pass,2158,AAA,431.6
32885,0-35,5,54.4,Pass,1034,AAA,206.8
38053,35-55,5,68.0,Pass,2445,AAA,489.0
...,...,...,...,...,...,...,...
2620947,0-35,9,88.9,Distinction,476,GGG,52.9
2645731,35-55,9,88.1,Distinction,893,GGG,99.2
2648187,0-35,9,76.7,Pass,312,GGG,34.7


In [31]:
final_df.isnull().sum()

age_band            0
n_assignments       0
mean_score         19
final_result        0
total_clicks        0
code_module         0
clicks_per_asmt     0
dtype: int64

In [32]:
final_df[final_df.isnull().any(axis=1)]

,age_band,n_assignments,mean_score,final_result,total_clicks,code_module,clicks_per_asmt
id_student,,,,,,,
186780,35-55,0,NaN,Withdrawn,30,BBB,inf
549713,35-55,0,NaN,Withdrawn,75,BBB,inf
554393,35-55,0,NaN,Fail,21,BBB,inf
606501,0-35,0,NaN,Withdrawn,9,BBB,inf
654422,0-35,0,NaN,Withdrawn,31,BBB,inf
555297,0-35,0,NaN,Withdrawn,155,DDD,inf
557247,0-35,0,NaN,Withdrawn,576,DDD,inf
557247,0-35,0,NaN,Withdrawn,576,DDD,inf
427248,35-55,0,NaN,Withdrawn,317,DDD,inf


In [33]:
#replace nan with mean value
final_df["mean_score"].fillna(72.83, inplace = True)

In [37]:
#replace "0s" with mean
final_df['n_assignments'] = np.where((final_df.n_assignments == 0), 7, final_df.n_assignments)

In [38]:
final_df['mean_score'] = np.where((final_df.mean_score == 0.0), 72.83, final_df.mean_score)

## scaling all feats. 

In [24]:
final_df.columns

Index(['age_band', 'n_assignments', 'mean_score', 'final_result',
       'total_clicks', 'code_module', 'clicks_per_asmt'],
      dtype='object')

In [25]:

NUM_FEATS = ['n_assignments', 'mean_score', 'total_clicks', 'clicks_per_asmt']
CAT_FEATS = ['age_band', 'final_result', 'code_module' ]
FEATS = NUM_FEATS + CAT_FEATS

In [26]:
numeric_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), 
                ('scaler', StandardScaler())])

In [27]:
categorical_transformer = \
Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [28]:
preprocessor = \
ColumnTransformer(transformers=[('num', numeric_transformer, NUM_FEATS),
                                ('cat', categorical_transformer, CAT_FEATS)])

In [29]:
preprocessor

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('num',
                                 Pipeline(memory=None,
                                          steps=[('imputer',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0)),
                                                 ('scaler',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                             

In [30]:
final_df_scaled = pd.DataFrame(data=preprocessor.fit_transform(final_df))

ValueError: Input contains infinity or a value too large for dtype('float64').

In [ ]:
final_df_scaled

In [ ]:
#final_all_num = final_df.merge(cat_dummies, left_index=True, right_index=True)

In [ ]:
#final_all_num

In [ ]:
#Dropping Num feats and dummies on final_df
#drop_col(final_df, FEATS)

In [ ]:
#cat_dummies

#blind_num_df


In [ ]:
#blind_num_df.to_csv('../data/processed/blind_num_df.csv', index=False)
#merge_final_cat.to_csv('../data/processed/cat_merge.csv', index=False)

# UMAP

In [ ]:
final_df_scaled = final_df_scaled.to_numpy()

In [ ]:
reducer = umap.UMAP(random_state=42)
reducer.fit(final_df_scaled.data)

In [ ]:
embedding = reducer.transform(final_df_scaled.data)

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1], cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP projection of OU Dataset -All Scaled-', fontsize=24);